In [1]:
import zipfile
import os
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import xml.etree.ElementTree as ET
import tensorflow as t
import torch
from sentence_transformers import SentenceTransformer, util

csv_file = '/home/abarovic/laneyREUTest/eclipse_platform.csv'
full_df = pd.read_csv(csv_file)

full_df.fillna(0,inplace=True)
df_sorted = full_df.sort_values(by='Created_time')
length=len(df_sorted)
split_index=int(.8*length)
df=df_sorted.iloc[:split_index] #df=training set

df.head()

df.info()

def is_duplicate(val):
    return 1 if val !=0.0 else 0
df['isDuplicate'] = df['Duplicated_issue'].apply(lambda x: is_duplicate(x))

#print(df['isDuplicate'])

import matplotlib.pyplot as plt

# Assuming 'df' is your DataFrame and 'isDuplicate' is the column indicating duplicates
duplicate_counts = df['isDuplicate'].value_counts()

# Check the contents of duplicate_counts
#print(duplicate_counts)

# Create a list of labels matching the length of duplicate_counts
# Ensure the labels match the values in the 'isDuplicate' column
labels = duplicate_counts.index.tolist()

# Define colors (length should match number of unique values in 'isDuplicate')
colors = ['#FFB6C1', '#D8BFD8'][:len(labels)]

# Plotting the pie chart
#plt.figure(figsize=(8, 6))
#plt.pie(duplicate_counts, labels=labels, autopct='%1.1f%%', startangle=140, colors=colors)
#plt.title('Percentage of Duplicates vs. Non-Duplicates')
#plt.axis('equal')
#plt.show()

df['Description'] = df['Description'].astype(str).fillna('')

#creating freeform textual data column
df['full_text_data']=df['Description']+df['Title']
#print(df['full_text_data'])
df['full_text_data']=df['full_text_data'].fillna('')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer
count_vectorizer = CountVectorizer(max_df=0.9, min_df=2, stop_words='english')

# Fit and transform the CountVectorizer on the text data
counts = count_vectorizer.fit_transform(df['full_text_data'])

count_vectorizer = CountVectorizer(max_df=0.9, min_df=2, stop_words='english')
counts = count_vectorizer.fit_transform(df['full_text_data'])

        # Fit LDA model
lda = LatentDirichletAllocation(n_components=7, max_iter=10,learning_method='online',learning_offset=50.,random_state=0)
lda.fit(counts)

feature_names = count_vectorizer.get_feature_names_out()
#for topic_idx, topic in enumerate(lda.components_):
#        print(f"Topic {topic_idx + 1}:")
#        print(" ".join([feature_names[i]
#                        for i in topic.argsort()[:-20 - 1:-1]]))
        
#        print()

######
#for alll columns in the df 
#compare each bug report to each topic 
#if
topic_distributions = lda.transform(counts)

df['topic'] = topic_distributions.argmax(axis=1)

# Create DataFrames for each topic
topic_dfs = []
for topic_idx in range(7):
    topic_df = df[df['topic'] == topic_idx].copy()
    topic_dfs.append(topic_df)

# Display the DataFrames for each topic
#for i, topic_df in enumerate(topic_dfs):
#    print(f"Topic {i}:")
#    print(topic_df['full_text_data'])
#    print()

#save each DataFrame to a CSV file
for i, topic_df in enumerate(topic_dfs):
    topic_df.to_csv(f'topic_{i}_bug_reports.csv', index=False)

topic_0_df=pd.read_csv('topic_0_bug_reports.csv')
topic_1_df=pd.read_csv('topic_1_bug_reports.csv')
topic_2_df=pd.read_csv('topic_2_bug_reports.csv')
topic_3_df=pd.read_csv('topic_3_bug_reports.csv')
topic_4_df=pd.read_csv('topic_4_bug_reports.csv')
topic_5_df=pd.read_csv('topic_5_bug_reports.csv')
topic_6_df=pd.read_csv('topic_6_bug_reports.csv')


#storaging them in seperate csv so when we do classifciaton on them its alll seperate 

def precisionRateAtK(df,k):
    #updates the row called Experimental_Duplicate_IDs to have the top k most similar bug ids
    top_k_recommendations_df=df
    matches=find_num_matches(top_k_recommendations_df) #finds relevant results 
    total=top_k_recommendations_df.shape[0]# Get the number of rows AKA total number of bugs
    precisionRateAtK=matches/total
    
    return precisionRateAtK
def recallRateAtK(df,k):
    #updates the row called Experimental_Duplicate_IDs to have the top k most similar bug ids
    top_k_recommendations_df=get_top_k_similar(df,k)
    matches=find_num_matches(top_k_recommendations_df) #finds relevant results 
    total=top_k_recommendations_df['isDuplicate'].sum()#finds num duplicates
    recallRateAtk=matches/total
    
    return recallRateAtk

import torch
from sentence_transformers import SentenceTransformer, util
import pandas as pd

def get_top_k_similar(df, k, model_name='paraphrase-MiniLM-L6-v2'):
    # Load the model
    model = SentenceTransformer(model_name)
    
    # Encode sentences and store the embeddings in the DataFrame
    df['embeddings'] = df['full_text_data'].apply(lambda x: model.encode(x, convert_to_tensor=True))
    
    # Convert embeddings to a tensor
    embeddings = torch.stack(df['embeddings'].tolist())
    
    # Calculate cosine similarity
    cosine_scores = util.pytorch_cos_sim(embeddings, embeddings)
    
    # Store the top k similar pairs
    top_k_similarities = {}
    for i in range(len(cosine_scores)):
        top_k_similarities[i] = sorted(
            [(j, cosine_scores[i][j].item()) for j in range(len(cosine_scores)) if i != j], 
            key=lambda x: x[1], 
            reverse=True
        )[:k]
    
    # Create a new column to store the experimental duplicate Issue IDs
    df['Experimental_Duplicate_IDs'] = df.index.map(
        lambda idx: [df.loc[pair[0], 'Issue_id'] for pair in top_k_similarities[idx]]
    )#this assigns the duplicates based on their ids not the index of the row 
    return df

import pandas as pd

def count_matches(row):
    return row['Duplicated_issue'] in row['Experimental_Duplicate_IDs']

def find_num_matches(df):
    return df.apply(count_matches, axis=1).sum()

topic_0_df.head()


2025-01-06 08:01:11.840177: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-06 08:01:11.843528: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-06 08:01:11.854624: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736150471.873256 3493147 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736150471.878902 3493147 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-06 08:01:11.898772: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

<class 'pandas.core.frame.DataFrame'>
Index: 68124 entries, 0 to 68123
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Issue_id          68124 non-null  int64  
 1   Priority          68124 non-null  object 
 2   Component         68124 non-null  object 
 3   Duplicated_issue  68124 non-null  float64
 4   Title             68124 non-null  object 
 5   Description       68124 non-null  object 
 6   Status            68124 non-null  object 
 7   Resolution        68124 non-null  object 
 8   Version           68124 non-null  object 
 9   Created_time      68124 non-null  object 
 10  Resolved_time     68124 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 6.2+ MB


/tmp/ipykernel_3493147/3647427725.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['isDuplicate'] = df['Duplicated_issue'].apply(lambda x: is_duplicate(x))
/tmp/ipykernel_3493147/3647427725.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Description'] = df['Description'].astype(str).fillna('')
/tmp/ipykernel_3493147/3647427725.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

,Issue_id,Priority,Component,Duplicated_issue,Title,Description,Status,Resolution,Version,Created_time,Resolved_time,isDuplicate,full_text_data,topic
0,217,P3,Team,0.0,UI BUG TO BE FILED (1GHQOGF),While minimizing an Eclipse running in Eclipse...,RESOLVED,INVALID,2.0,2001-10-10 21:38:00 -0400,2001-10-23 23:47:22 -0400,0,While minimizing an Eclipse running in Eclipse...,0
1,1533,P2,Debug,0.0,Feature: notification of changing variables (1...,DW (12/8/00 4:03:27 PM); \tThe old VAME debugg...,VERIFIED,FIXED,2.0,2001-10-10 22:14:00 -0400,2002-02-04 13:16:45 -0500,0,DW (12/8/00 4:03:27 PM); \tThe old VAME debugg...,0
2,1850,P3,UI,0.0,WALKBACK - EditableTable (1G5T6MC),Debugger Stack Trace Report:; ; Thread[main;5;...,RESOLVED,FIXED,2.0,2001-10-10 22:20:00 -0400,2001-11-13 15:30:56 -0500,0,Debugger Stack Trace Report:; ; Thread[main;5;...,0
3,2650,P3,UI,0.0,[JFace Text] TextViewer#setVisibleRegion doesn...,Create a TextViewer; give it a string for its ...,RESOLVED,WONTFIX,2.0,2001-10-10 22:40:00 -0400,2009-08-30 02:31:49 -0400,0,Create a TextViewer; give it a string for its ...,0
4,2760,P3,UI,0.0,Workbench resizing crash (stack overflow) (1GI...,Self hosting Eclipse; my target workbench cras...,RESOLVED,INVALID,2.0,2001-10-10 22:43:00 -0400,2002-01-08 15:59:57 -0500,0,Self hosting Eclipse; my target workbench cras...,0


In [2]:
import torch
from sentence_transformers import SentenceTransformer, util
import pandas as pd

def get_top_k_similar_v2(df, k, model_name='paraphrase-MiniLM-L6-v2'):
    # Load the model
    model = SentenceTransformer(model_name)
    
    # Encode sentences and store the embeddings in the DataFrame
    df['embeddings'] = df['full_text_data'].apply(lambda x: model.encode(x, convert_to_tensor=True))
    
    # Convert embeddings to a tensor
    embeddings = torch.stack(df['embeddings'].tolist())
    
    # Calculate cosine similarity
    cosine_scores = util.pytorch_cos_sim(embeddings, embeddings)
    
    # Store the top k similar pairs
    top_k_similarities = {}
    for i in range(len(cosine_scores)):
        similarities = [(j, cosine_scores[i][j].item()) for j in range(len(cosine_scores)) if i != j]
        top_k_similarities[i] = sorted(similarities, key=lambda x: x[1], reverse=True)[:k]
    
    # Create a new column to store the experimental duplicate Issue IDs
    def get_experimental_duplicates(idx):
        similar_pairs = top_k_similarities[idx]
        if all(score < 0.95 for _, score in similar_pairs):
            return [0]
        else:
            return [df.loc[pair[0], 'Issue_id'] for pair in similar_pairs]
    
    df['Experimental_Duplicate_IDs'] = df.index.map(get_experimental_duplicates)
    
    return df

# Initialize an empty list to store recall values for each topic
all_topic_vals_precision = []
all_topic_vals_recall = []

for topic_num in range(7):
    precision_vals = []
    recall_vals = []
    
    topic_df = globals()[f'topic_{topic_num}_df']  # Dynamically get the DataFrame for the current topic
    
    # Calculate precision and recall values at different k-values
    for k in [5, 10, 15, 20]:
        topic_df = get_top_k_similar_v2(topic_df, k)
        precision = precisionRateAtK(topic_df, k)
        recall = recallRateAtK(topic_df, k)
        
        precision_vals.append(precision)
        recall_vals.append(recall)
        
        print(f"Precision for topic {topic_num} at {k} is {precision}")
        print(f"Recall for topic {topic_num} at {k} is {recall}")
    
    # Append precision and recall values for the current topic
    all_topic_vals_precision.append(precision_vals)
    all_topic_vals_recall.append(recall_vals)



Precision for topic 0 at 5 is 0.8696808510638298
Recall for topic 0 at 5 is 0.13095238095238096
Precision for topic 0 at 10 is 0.8696808510638298
Recall for topic 0 at 10 is 0.1488095238095238
Precision for topic 0 at 15 is 0.8696808510638298
Recall for topic 0 at 15 is 0.1488095238095238
Precision for topic 0 at 20 is 0.8696808510638298
Recall for topic 0 at 20 is 0.16071428571428573
Precision for topic 1 at 5 is 0.8414634146341463
Recall for topic 1 at 5 is 0.20052770448548812
Precision for topic 1 at 10 is 0.8414634146341463
Recall for topic 1 at 10 is 0.21635883905013192
Precision for topic 1 at 15 is 0.8421807747489239
Recall for topic 1 at 15 is 0.23218997361477572
Precision for topic 1 at 20 is 0.8421807747489239
Recall for topic 1 at 20 is 0.23482849604221637
Precision for topic 2 at 5 is 0.8257835922594713
Recall for topic 2 at 5 is 0.17920918367346939
Precision for topic 2 at 10 is 0.8258381030253475
Recall for topic 2 at 10 is 0.21747448979591838
Precision for topic 2 at 15 

In [4]:
import numpy as np

# Calculate averages across all topics
average_precision_at_k = np.mean(all_topic_vals_precision, axis=0)
average_recall_at_k = np.mean(all_topic_vals_recall, axis=0)
mean_average_precision = np.mean(average_precision_at_k)  # Calculate mAP

# Print all values in the arrays for precision and recall
print("All Avg Precision Values at Each k:")
for i, precision_value in enumerate(average_precision_at_k, start=1):
    print(f"Value {i}: {precision_value:.4f}")

print("\nAll Avg Recall Values at Each k:")
for i, recall_value in enumerate(average_recall_at_k, start=1):
    print(f"Value {i}: {recall_value:.4f}")

# Print clean output for specific k values
k_values = [5, 10, 15, 20]
print("\nSummary for Specific k Values:")
for idx, k in enumerate(k_values):
    print(f"Recall Rate @ {k} is {average_recall_at_k[idx]:.4f}")
    print(f"Precision Rate @ {k} is {average_precision_at_k[idx]:.4f}")

print(f"\nMean Average Precision (mAP) is {mean_average_precision:.4f}")


All Avg Precision Values at Each k:
Value 1: 0.7951
Value 2: 0.7952
Value 3: 0.7953
Value 4: 0.7955

All Avg Recall Values at Each k:
Value 1: 0.1513
Value 2: 0.1795
Value 3: 0.1931
Value 4: 0.2038

Summary for Specific k Values:
Recall Rate @ 5 is 0.1513
Precision Rate @ 5 is 0.7951
Recall Rate @ 10 is 0.1795
Precision Rate @ 10 is 0.7952
Recall Rate @ 15 is 0.1931
Precision Rate @ 15 is 0.7953
Recall Rate @ 20 is 0.2038
Precision Rate @ 20 is 0.7955

Mean Average Precision (mAP) is 0.7953
